In [1]:
import torch
torch.cuda.empty_cache()

In [ ]:
# 🔧 FIX PIL/matplotlib DLL issues on Windows
!pip uninstall -y pillow matplotlib
!pip install pillow==9.5.0 matplotlib==3.7.2

print("✅ PIL/matplotlib compatibility fixed!")


In [ ]:
# Install required packages with fixed Pillow version
!pip install datasets soundfile speechbrain
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate
!pip install requests

# Fix PIL/matplotlib DLL issues on Windows
!pip uninstall -y pillow matplotlib
!pip install pillow==9.5.0 
!pip install matplotlib==3.7.2

!pip install noisereduce librosa

# Check GPU availability
!nvidia-smi

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached speechbrain-1.0.3-py3-none-any.whl.metadata (24 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached HyperPyYAML-1.2.2-py3-none-any.whl.metadata (7.6 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached sentencepiece-0.2.0-cp39-cp39-win_amd64.whl.metadata (8.3 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached multidict-6.4.4-cp39-cp39-win_amd64.whl.metadata (5.5 kB)
  Using cached sympy-1.13.1-py3-none-any.whl

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\PC\AppData\Local\Temp\pip-req-build-ljh70rui'


  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.7.0-py3-none-any.whl (362 kB)
  Using cached noisereduce-3.0.3-py3-none-any.whl.metadata (14 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached numba-0.60.0-cp39-cp39-win_amd64.whl.metadata (2.8 kB)
  Using cached scikit_learn-1.6.1-cp39-cp39-win_amd64.whl.metadata (15 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.5.0.post1-cp39-cp39-win_amd64.whl.metadata (5.6 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached llvmlite-0.43.0-cp39-cp39-win_amd64.whl.metadata (4.9 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.

In [4]:
# Import required libraries
import json
import os
import tempfile
import torch
# import matplotlib.pyplot as plt  # Removed due to PIL DLL issues
from collections import defaultdict
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from IPython.display import Audio
import soundfile as sf

from transformers import (
    SpeechT5Processor,
    SpeechT5ForTextToSpeech,
    SpeechT5HifiGan,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import Dataset, Audio as AudioFeature

# For denoising and trimming
import noisereduce as nr
import librosa

# Import LocalStrategy to avoid symlink errors on Windows
from speechbrain.utils.fetching import LocalStrategy
from speechbrain.pretrained import EncoderClassifier

# Load SpeechT5 model + processor
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

# Load Pashto dataset
def load_pashto_dataset(json_file_path: str) -> Dataset:
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    dataset_dict = {
        'id': [],
        'audio_url': [],
        'text': [],
        'gender': [],
        'accent': [],
        'speaker_id': []
    }
    
    for item in data:
        dataset_dict['id'].append(item['id'])
        dataset_dict['audio_url'].append(item['file_url'])
        dataset_dict['text'].append(item['sentence'])
        dataset_dict['gender'].append(item['gender'])
        dataset_dict['accent'].append(item['accent'])
        dataset_dict['speaker_id'].append(f"{item['gender']}_{item['accent'].replace(' ', '_')}")
    
    return Dataset.from_dict(dataset_dict)

dataset_path = r"C:\Users\PC\Music\jj\new3.json"  # Replace with your actual path
dataset = load_pashto_dataset(dataset_path)

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [4]:
# 🎯 LIMIT DATASET TO 1000 SAMPLES FOR FASTER TRAINING
print(f"📋 Original dataset size: {len(dataset)} samples")
dataset = dataset.select(range(min(1000, len(dataset))))
print(f"🎯 Dataset LIMITED to {len(dataset)} samples for faster training and testing")
print(f"✅ This ensures we only process 1000 audio files maximum")


📋 Original dataset size: 10000 samples
🎯 Dataset LIMITED to 1000 samples for faster training and testing
✅ This ensures we only process 1000 audio files maximum


In [5]:
# 🎯 FIRST: Limit dataset to 1000 samples BEFORE expensive audio processing
print(f"📋 Original dataset size: {len(dataset)} samples")
dataset = dataset.select(range(min(1000, len(dataset))))
print(f"🎯 Dataset LIMITED to {len(dataset)} samples for faster training")

# Load audio files from local folder
LOCAL_AUDIO_DIR = r"C:\Users\PC\Downloads\AudioFiles"

def load_local_audio(example: Dict[str, Any]) -> Dict[str, Any]:
    filename = os.path.basename(example['audio_url'])
    local_path = os.path.join(LOCAL_AUDIO_DIR, filename)
    
    if os.path.isfile(local_path):
        try:
            audio_array, sample_rate = sf.read(local_path)
            example['audio'] = {'array': audio_array, 'sampling_rate': sample_rate}
        except Exception as e:
            example['audio'] = None
    else:
        example['audio'] = None
    
    return example

dataset = dataset.map(load_local_audio)
dataset = dataset.filter(lambda x: x['audio'] is not None)
print(f"📊 Final dataset size after filtering: {len(dataset)} audio files")

dataset = dataset.cast_column("audio", AudioFeature(sampling_rate=16000))

# Enhanced text processing with prosodic-aware tokenization for Pashto TTS
import re
from typing import List, Tuple

# Add Pashto-specific replacements with prosodic-aware processing
def cleanup_text(inputs: Dict[str, Any]) -> Dict[str, Any]:
    text = inputs["text"]
    
    # Pashto character replacements
    replacements = [
        ('ښ', 'ښ'),  # Ensure consistent shin
        ('ګ', 'ګ'),  # Ensure consistent gaf
        ('ځ', 'ځ'),  # Ensure consistent dzal
        ('څ', 'څ'),  # Ensure consistent tse
        ('ډ', 'ډ'),  # Ensure consistent dal
        ('ړ', 'ړ'),  # Ensure consistent re
        ('ټ', 'ټ'),  # Ensure consistent te
        ('ږ', 'ږ'),  # Ensure consistent zhge
        ('ڼ', 'ڼ'),  # Ensure consistent noon
        ('ې', 'ې'),  # Ensure consistent ye
        ('ۍ', 'ۍ'),  # Ensure consistent ye
        
        # Remove extra whitespace
        (r'\s+', ' '),
        
        # Normalize punctuation
        ('،', '،'),
        ('؟', '؟'),
        ('؛', '؛'),
        
        # Remove problematic characters
        ('\u200c', ''),  # Zero-width non-joiner
        ('\u200d', ''),  # Zero-width joiner
        ('\u200e', ''),  # Left-to-right mark
        ('\u200f', ''),  # Right-to-left mark
    ]
    
    for src, dst in replacements:
        text = text.replace(src, dst)
    
    # Create phrase-aware text for better prosody
    phrase_text = create_phrase_aware_text(text)
    
    inputs["normalized_text"] = text
    inputs["phrase_text"] = phrase_text
    return inputs

def create_phrase_aware_text(text: str) -> str:
    """
    Create phrase-aware text that maintains prosodic boundaries for Pashto TTS.
    This addresses the choppy audio issue by preserving natural speech rhythm.
    """
    # Pashto prosodic boundary markers
    # Major boundaries (sentence level)
    text = re.sub(r'[؟!۔]', ' <MAJOR_BREAK> ', text)
    
    # Medium boundaries (phrase level) 
    text = re.sub(r'[،؛]', ' <PHRASE_BREAK> ', text)
    
    # Minor boundaries (word group level) - every 2-3 words for Pashto rhythm
    words = text.split()
    phrase_words = []
    
    for i, word in enumerate(words):
        phrase_words.append(word)
        # Add minor breaks every 2-3 words to maintain natural Pashto rhythm
        if (i + 1) % 3 == 0 and i < len(words) - 1:
            if word not in ['<MAJOR_BREAK>', '<PHRASE_BREAK>']:
                phrase_words.append('<MINOR_BREAK>')
    
    phrase_text = ' '.join(phrase_words)
    
    # Clean up multiple breaks
    phrase_text = re.sub(r'(<[^>]+>\s*){2,}', r'<PHRASE_BREAK> ', phrase_text)
    phrase_text = re.sub(r'\s+', ' ', phrase_text).strip()
    
    return phrase_text

dataset = dataset.map(cleanup_text)

# Enhanced tokenizer setup with prosodic tokens
tokenizer = processor.tokenizer

# Add prosodic boundary tokens for better speech flow
prosodic_tokens = ['<MAJOR_BREAK>', '<PHRASE_BREAK>', '<MINOR_BREAK>', '<STRESS>', '<UNSTRESS>']

def extract_all_chars(batch):
    all_text = " ".join(batch["text"] + batch["phrase_text"])
    chars = list(set(all_text))
    return {"vocab": [chars], "all_text": [all_text]}

vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = set(tokenizer.get_vocab().keys())

# Add missing characters and prosodic tokens
missing_chars = dataset_vocab - tokenizer_vocab
all_new_tokens = list(missing_chars) + prosodic_tokens

if all_new_tokens:
    tokenizer.add_tokens(all_new_tokens)
    processor.tokenizer = tokenizer
    model.resize_token_embeddings(len(tokenizer))
    
print(f"✅ Added {len(all_new_tokens)} new tokens including prosodic markers")
print(f"📝 Prosodic tokens added: {prosodic_tokens}")
print(f"🔤 Updated vocabulary size: {len(tokenizer)}")
print("🎵 This phrase-level approach should reduce choppiness in Pashto TTS!")


📋 Original dataset size: 1000 samples
🎯 Dataset LIMITED to 1000 samples for faster training


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

📊 Final dataset size after filtering: 1000 audio files


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


✅ Added 63 new tokens including prosodic markers
📝 Prosodic tokens added: ['<MAJOR_BREAK>', '<PHRASE_BREAK>', '<MINOR_BREAK>', '<STRESS>', '<UNSTRESS>']
🔤 Updated vocabulary size: 144
🎵 This phrase-level approach should reduce choppiness in Pashto TTS!


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# Load speaker model ONCE outside the function
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example["audio"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get("phrase_text", example["normalized_text"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )
    example["labels"] = processed["labels"][0]
    example["input_ids"] = processed["input_ids"]
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed["input_ids"], tokenizer)
    example["prosodic_features"] = prosodic_features
    
    return example

def create_prosodic_features(input_ids: torch.Tensor, tokenizer) -> torch.Tensor:
    """Create prosodic features based on break tokens to guide TTS synthesis"""
    prosodic_features = torch.zeros(len(input_ids))
    
    # Get token IDs for prosodic markers
    break_token_ids = {
        tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0] if '<MAJOR_BREAK>' in tokenizer.get_vocab() else -1,
        tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0] if '<PHRASE_BREAK>' in tokenizer.get_vocab() else -1,
        tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0] if '<MINOR_BREAK>' in tokenizer.get_vocab() else -1,
    }
    break_token_ids.discard(-1)  # Remove invalid tokens
    
    for i, token_id in enumerate(input_ids):
        if token_id.item() in break_token_ids:
            # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
            if '<MAJOR_BREAK>' in tokenizer.get_vocab() and token_id == tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]:
                prosodic_features[i] = 1.0
            elif '<PHRASE_BREAK>' in tokenizer.get_vocab() and token_id == tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]:
                prosodic_features[i] = 0.7
            elif '<MINOR_BREAK>' in tokenizer.get_vocab() and token_id == tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]:
                prosodic_features[i] = 0.3
    
    return prosodic_features

# Now run dataset mapping - will be ~100x faster
dataset = dataset.map(
    prepare_dataset,
    remove_columns=["audio", "audio_url", "text", "gender", "accent", "id"],
)

In [41]:
# Enhanced Data Collator with Prosodic Features
@dataclass
class TTSDataCollatorEnhanced:
    processor: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Extract input components
        input_ids = [{"input_ids": f["input_ids"]} for f in features]
        label_features = [{"input_values": f["labels"]} for f in features]
        speaker_features = [f["speaker_embeddings"] for f in features]
        
        # Handle prosodic features
        prosodic_features = [f.get("prosodic_features", torch.zeros(len(f["input_ids"]))) for f in features]
        
        # Pad sequences
        batch = self.processor.pad(
            input_ids=input_ids,
            labels=label_features,
            return_tensors="pt",
        )

        # Mask labels for padding
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )
        del batch["decoder_attention_mask"]

        # Handle reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor([len(f["input_values"]) for f in label_features])
            target_lengths = target_lengths.new([
                length - length % model.config.reduction_factor
                for length in target_lengths
            ])
            max_length = max(target_lengths).item()
            batch["labels"] = batch["labels"][:, :max_length]

        # Add speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)
        
        # Pad prosodic features to match input sequence length
        max_seq_len = batch["input_ids"].size(1)
        padded_prosodic = []
        
        for prosodic in prosodic_features:
            if len(prosodic) < max_seq_len:
                # Pad with zeros
                padded = torch.cat([prosodic, torch.zeros(max_seq_len - len(prosodic))])
            else:
                # Truncate if too long
                padded = prosodic[:max_seq_len]
            padded_prosodic.append(padded)
        
        batch["prosodic_features"] = torch.stack(padded_prosodic)
        
        return batch

# Initialize enhanced data collator
data_collator = TTSDataCollatorEnhanced(processor=processor)
print("Enhanced data collator initialized with prosodic feature support!")

In [42]:
import torch
print("CUDA Available:", torch.cuda.is_available())  # Should print True if GPU is available

CUDA Available: True


In [43]:
import torch
import logging
import os
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, SpeechT5ForTextToSpeech

logging.basicConfig(level=logging.INFO)

# ADD THIS LINE - Split the dataset first
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Training dataset size: {len(dataset['train'])}")
print(f"Validation dataset size: {len(dataset['test'])}")

# Calculate expected steps
samples_per_step = 4  # batch_size * gradient_accumulation_steps  
steps_per_epoch = len(dataset['train']) // samples_per_step
print(f"Expected steps per epoch: {steps_per_epoch}")

# Load model weights from checkpoint but start training from scratch
checkpoint_base = r"C:\Users\PC\speecht5_tts_pashto"
latest_checkpoint = None

if os.path.isdir(checkpoint_base):
    ckpts = [d for d in os.listdir(checkpoint_base)
             if d.startswith("checkpoint-") and os.path.isdir(os.path.join(checkpoint_base, d))]
    if ckpts:
        latest = max(ckpts, key=lambda x: int(x.split("-", 1)[1]))
        latest_checkpoint = os.path.join(checkpoint_base, latest)
        print(f"Loading model weights from: {latest_checkpoint}")
        
        # Load only the model weights, not the training state
        model = SpeechT5ForTextToSpeech.from_pretrained(latest_checkpoint)
        model = model.to('cuda')
        print(f"Model weights loaded with vocab size: {model.config.vocab_size}")
    else:
        print("No checkpoints found, using current model")
        model = model.to('cuda')
else:
    print("No checkpoint directory found, using current model")
    model = model.to('cuda')

# Fresh training arguments - no resume_from_checkpoint
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_base,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    warmup_steps=100,
    num_train_epochs=5,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    gradient_checkpointing=True,
    fp16=False,
    logging_steps=200,
    report_to=[],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    processing_class=processor,
)

print("Starting fresh training with loaded model weights...")
trainer.train()
print("Training completed successfully!")

Training dataset size: 9000
Validation dataset size: 1000
Expected steps per epoch: 2250
Loading model weights from: C:\Users\PC\speecht5_tts_pashto\checkpoint-11250
Model weights loaded with vocab size: 170
Starting fresh training with loaded model weights...


Step,Training Loss,Validation Loss
200,0.472000,0.551322
400,0.470300,0.552072
600,0.464300,0.550848
800,0.471000,0.548769
1000,0.469500,0.547081
1200,0.469900,0.553332
1400,0.466900,0.547237
1600,0.464100,0.543017
1800,0.466700,0.546820
2000,0.467500,0.545039


Training completed successfully!


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [ ]:
# 🔧 FIXED SPEAKER EMBEDDING AND DATASET PREPARATION - Resolved .item() error
device = "cuda" if torch.cuda.is_available() else "cpu"
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_cache = os.path.join(tempfile.gettempdir(), spk_model_name)

print(f"💻 Using device: {device}")
print("🔄 Loading speaker model...")

# GLOBAL MODEL - loaded once
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=speaker_cache,
    local_strategy=LocalStrategy.COPY
)
print("✅ Speaker model loaded successfully!")

def create_speaker_embedding(waveform: List[float]) -> Any:
    # Use the already loaded global model
    with torch.no_grad():
        embeddings = speaker_model.encode_batch(torch.tensor(waveform).to(device))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        return embeddings.squeeze().cpu().numpy()

def create_prosodic_features(input_ids, tokenizer) -> torch.Tensor:
    \"\"\"🎵 FIXED: Create prosodic features based on break tokens to guide TTS synthesis\"\"\"
    # Convert to tensor if it's a list, otherwise keep as is
    if isinstance(input_ids, list):
        input_ids_tensor = torch.tensor(input_ids)
    else:
        input_ids_tensor = input_ids
    
    prosodic_features = torch.zeros(len(input_ids_tensor))
    
    # Get token IDs for prosodic markers safely
    break_token_ids = {}
    try:
        if '<MAJOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['major'] = tokenizer.encode('<MAJOR_BREAK>', add_special_tokens=False)[0]
        if '<PHRASE_BREAK>' in tokenizer.get_vocab():
            break_token_ids['phrase'] = tokenizer.encode('<PHRASE_BREAK>', add_special_tokens=False)[0]
        if '<MINOR_BREAK>' in tokenizer.get_vocab():
            break_token_ids['minor'] = tokenizer.encode('<MINOR_BREAK>', add_special_tokens=False)[0]
    except (IndexError, KeyError):
        # If encoding fails, return zeros (no prosodic features)
        return prosodic_features
    
    for i, token_id in enumerate(input_ids_tensor):
        # 🔧 FIXED: Handle both tensor and int token_id properly
        token_val = token_id.item() if hasattr(token_id, 'item') else int(token_id)
        
        # Assign prosodic strength: major=1.0, phrase=0.7, minor=0.3
        if 'major' in break_token_ids and token_val == break_token_ids['major']:
            prosodic_features[i] = 1.0
        elif 'phrase' in break_token_ids and token_val == break_token_ids['phrase']:
            prosodic_features[i] = 0.7
        elif 'minor' in break_token_ids and token_val == break_token_ids['minor']:
            prosodic_features[i] = 0.3
    
    return prosodic_features

def prepare_dataset(example: Dict[str, Any]) -> Dict[str, Any]:
    audio = example[\"audio\"]
    
    # Use phrase_text for better prosodic flow instead of normalized_text
    text_to_use = example.get(\"phrase_text\", example[\"normalized_text\"])
    
    processed = processor(
        text=text_to_use,
        audio_target=audio[\"array\"],
        sampling_rate=audio[\"sampling_rate\"],
        return_attention_mask=False,
    )
    example[\"labels\"] = processed[\"labels\"][0]
    example[\"input_ids\"] = processed[\"input_ids\"]
    example[\"speaker_embeddings\"] = create_speaker_embedding(audio[\"array\"])
    
    # Add prosodic features based on break tokens
    prosodic_features = create_prosodic_features(processed[\"input_ids\"], tokenizer)
    example[\"prosodic_features\"] = prosodic_features
    
    return example

print(\"🚀 Starting dataset preparation with 1000 samples...\")


In [44]:
# Save model and processor
trainer.save_model("./speecht5_tts_pashto_final")
processor.save_pretrained("./speecht5_tts_pashto_final")

# Inference with the trained model
import torch
import librosa
import soundfile as sf
from IPython.display import Audio, display

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
model = SpeechT5ForTextToSpeech.from_pretrained("./speecht5_tts_pashto_final")

# Load speaker embeddings (use the same one from training)
import datasets
embeddings_dataset = datasets.load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Move to GPU if available
if torch.cuda.is_available():
   model = model.to('cuda')
   vocoder = vocoder.to('cuda')
   speaker_embeddings = speaker_embeddings.to('cuda')

test_text = "صبر، د زغم نښه ده!"  # "Hello brother, how are you?"
inputs = processor(text=test_text, return_tensors="pt")

# Move inputs to same device as model
if torch.cuda.is_available():
   inputs = {k: v.to('cuda') for k, v in inputs.items()}

# Generate speech
with torch.no_grad():
   spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
   speech = vocoder(spectrogram)

# Move back to CPU for processing
waveform = speech.squeeze().cpu().numpy()
sr = 16000

# Trim silence
trimmed, _ = librosa.effects.trim(waveform)

# Playback and save
display(Audio(trimmed, rate=sr))
sf.write("pashto_output_trimmed.wav", trimmed, samplerate=sr)
print("Audio saved as 'pashto_output_trimmed.wav'")

Audio saved as 'pashto_output_trimmed.wav'


In [ ]:
# 🎵 COMPARISON: Word-level vs Phrase-level Tokenization for Pashto TTS
# This demonstrates why phrase-level tokenization reduces choppiness

# Test with the same Pashto sentence using both approaches
test_sentence = "زه ډېر خوشحاله یم چې دلته یم، ښه راغلاست!"  # "I am very happy to be here, welcome!"

print("🔍 TOKENIZATION COMPARISON FOR PASHTO TTS:")
print("=" * 60)

# Original word-level approach (choppy)
word_tokens = processor.tokenizer.tokenize(test_sentence)
print("❌ Word-level tokens (causes choppiness):")
print(f"   {word_tokens}")
print(f"   Length: {len(word_tokens)} tokens")

# Our improved phrase-level approach
phrase_sentence = create_phrase_aware_text(test_sentence)
phrase_tokens = processor.tokenizer.tokenize(phrase_sentence)
print(f"\n✅ Phrase-level tokens (better prosody):")
print(f"   {phrase_tokens}")
print(f"   Length: {len(phrase_tokens)} tokens")

print(f"\n📝 Original: {test_sentence}")
print(f"📝 With breaks: {phrase_sentence}")

# Generate audio with improved approach
inputs = processor(text=phrase_sentence, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.to('cuda') for k, v in inputs.items()}

with torch.no_grad():
    spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
    speech = vocoder(spectrogram)

# Process and save improved audio
waveform = speech.squeeze().cpu().numpy()
trimmed, _ = librosa.effects.trim(waveform)

# Apply light noise reduction for cleaner output
if len(trimmed) > 1000:  # Only if audio is long enough
    trimmed = nr.reduce_noise(y=trimmed, sr=16000, prop_decrease=0.8)

display(Audio(trimmed, rate=16000))
sf.write("pashto_improved_prosody.wav", trimmed, samplerate=16000)

print("\n🎉 SOLUTION SUMMARY:")
print("✅ Phrase-level tokenization maintains natural Pashto rhythm")
print("✅ Prosodic break tokens guide speech synthesis timing") 
print("✅ Reduces choppy artifacts from word-level boundaries")
print("✅ Audio saved as 'pashto_improved_prosody.wav'")
print("\n💡 The key insight: Pashto TTS needs phrase-level context, not just word-level!")
